# Creating models step-by-step

In [17]:
import os
import json
import numpy as np
import pandas as pd
import collections
import sys
import seaborn as sns
import copy
import subprocess

from libchebipy._chebi_entity import ChebiEntity
from Bio import Entrez, SeqIO
import reframed
from reframed.io.sbml import parse_gpr_rule

sys.path.append('../functions')

import translation_dicts
import general_functions as general_func
import EGC as EGC
import MAG_environments

# Prepare data 

In [18]:
! mkdir ../output/
! mkdir ../output/MAGs_fasta/

%run -i "./python_scripts_process_data/1. Processing MAG data - From genbank to fasta.py"

%run -i "./python_scripts_process_data/2. Defining metabolite classes.py" ## Made cellb to simple sugar

%run -i "./python_scripts_process_data/3. MAG-community association.py"

%run -i "./python_scripts_process_data/4. Community production.py"


# Build genome-scale metabolic models

## Create models without constraints

In [19]:
! mkdir ../output/GEMs
! mkdir ../output/GEMs/GEMs_no_constraints/

In [20]:
! source activate MAGs_RecAndAn2; ./bash_scripts/make_GEMs2.sh

## Create models with constraints

#### Create constraints

In [21]:
! mkdir ../output/soft_constraints/

In [22]:
%run -i "python_scripts/1. create soft constraints.py"

In [23]:
SC_media_test_old = pd.read_csv("../output_30_08_24/soft_constraints/SC_CD_X.tsv",
                                          sep="\t",
                                          header=None)
SC_media_test_old

In [24]:
SC_media_test_new = pd.DataFrame(pd.Series(soft_constraints_new["CD_X"])).reset_index()
SC_media_test_new

#### Create models 

In [25]:
! mkdir ../output/GEMs/GEMs_intermediate/


In [26]:
! source activate MAGs_RecAndAn2; ./bash_scripts/make_GEMs_soft_constraints.sh 0.1


## Select models

We have different sets of reconstructed models.

- **no_constr**: no soft constraints included. This is the ideal, but not all metabolites were produced.
- **constr0_1**: Will likely add reactions needed to support the phenotype, but it is a biased approach. May overpredict the amount of models carrying the ability. 

Strategy

1. Find the difference between the models
2. **Selection**: The models that required the least amount of changes (symmetric difference) to acquire the desired phenotype.

In [27]:
! mkdir ../output/GEMs/GEMs_intermediate/GEMs_adapt/

In [28]:
%run -i "python_scripts/2. Select models.py"

#### Showing that models are similar to the models used for analysis after rerunning the code

In [29]:
directory = os.fsencode("../output_30_08_24/GEMs/GEMs_intermediate/GEMs_adapt/")
difference = {}


for file in os.listdir(directory):

    filename = os.fsdecode(file)
    print(filename[:-4])
    difference[filename[:-4]] = {}    
    if filename.endswith(".xml"): 
        model = reframed.load_cbmodel("../output_30_08_24/GEMs/GEMs_intermediate/GEMs_adapt/"+filename)
        difference[filename[:-4]]["original"] = set(model.reactions) -set(GEMs_dict["adapt"][filename[:-4]].reactions)
        difference[filename[:-4]]["new"] = set(GEMs_dict["adapt"][filename[:-4]].reactions)-set(model.reactions)
        

**Differences are mostly in redundant reactions in the BiGG database...**

In [30]:
pd.DataFrame(difference).T.sort_values("new",ascending=False)

## Fixing acetate, acetaldehyde, and O2 transporters 

####  Find transporters

In [31]:
! mkdir ../output/transporters

In [32]:
! source activate MAGs_RecAndAn2; export PATH=$PATH:/usr/local/ncbi/blast/bin/; ./bash_scripts/acetate_transport_predict.sh

#### Fixing transporters

In [33]:
! mkdir ../output/GEMs/GEMs_intermediate/GEMs_ACt2r/

In [34]:
%run -i "python_scripts/3. Fixing acetate acetaldehyde and O2 transporters.py"

In [35]:
directory = os.fsencode("../output_30_08_24/GEMs/GEMs_intermediate/GEMs_ACt2r/")
for file in os.listdir(directory):
    
    filename = os.fsdecode(file)
    if filename.endswith(".xml"): 
        model_old = reframed.load_cbmodel("../output_30_08_24/GEMs/GEMs_intermediate/GEMs_ACt2r/"+filename)
        
        difference[filename[:-4]] = set(model_old.reactions).symmetric_difference(set(GEMs_dict3[filename[:-4]].reactions))
        

In [36]:
pd.Series({mag:len(set_) for mag,set_ in difference.items()}).sort_values().tail(20)

## Gap-filling with media - creating media 

This part builds on the reconstruction with soft constraints. The models selected from that process are further used here. 


Strategy:
- **Make media based on..**
    - **product prediction of models able to grow**
    - **Filter by CHEBI class**

In [37]:
! mkdir ../output/gapfill_media

In [38]:
%run -i "python_scripts/4. Create media for gapfilling.py"

##### Check that everything is as expected

In [39]:
media_total_df_old = pd.read_csv("../output_30_08_24/gapfill_media/gapfill_media.tsv",sep="\t")
media_total_df = pd.concat(media_dfs).reset_index(drop=True)

In [58]:
for community_id in media_total_df.medium.unique():
    print(community_id)
    compounds_medium = set(media_total_df[media_total_df.medium==community_id].compound.values)
    compounds_medium_prev = set(media_total_df_old[media_total_df_old.medium==community_id].compound.values)
    
    symm_diff = compounds_medium.symmetric_difference(compounds_medium_prev)
    if len(symm_diff)>0:
        print(symm_diff)

## Gapfill models

In [53]:
! mkdir ../output/GEMs/GEMs_final

In [54]:
! source activate MAGs_RecAndAn2; ./bash_scripts/gapfill_GEMs_media.sh


## Final changes

#### Load models

In [55]:
GEMs_adapt_media = {}

directory = os.fsencode("../output/GEMs/GEMs_final/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)

    GEMs_adapt_media[filename[:-4]]= reframed.load_cbmodel("../output/GEMs/GEMs_final/"+filename)


In [56]:
for MAG, model in GEMs_adapt_media.items():
    
    if "R_EX_2h3mb_e" in model.reactions:
        print(MAG)     
        rxns = model.get_metabolite_reactions("M_2h3mb_e")
        model.remove_reactions(rxns)
        model.remove_metabolite("M_2h3mb_e")
        model.update()
        GEMs_adapt_media[MAG]=model
        
        

In [57]:
for MAG,model in GEMs_adapt_media.items():
    reframed.save_cbmodel(model,filename="../output/GEMs/GEMs_final/"+MAG+".xml")